In [ ]:
import json
import nltk
import inflect
import tqdm
import time
import requests 
from bs4 import BeautifulSoup as bs
import gensim
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
import numpy as np
import collections
import json
import sys
sys.path.insert(0, "/Users/Lionel/Documents/Ada/Project/python")
import clean_recipes_datasets as clean
engine = inflect.engine()

In [ ]:
# Load the recipes for which we have the quantity for each ingredients 
with open('../generated/1m_quant_usda_recipes.json') as json_data:
    quantities_recipes = json.load(json_data)

# Table of conversion for the unit found in 1M
with open('../generated/convert_gr.json') as json_data:
    convert_gr = json.load(json_data)
    
# Mapping from ingredient to USDA id
with open('../generated/ing_usda_mapping_high_score_3d.json') as json_data:
    mapping_usda_id = json.load(json_data)
    
nutrients_mapping = pd.read_hdf("../generated/nut_data_3d.h5", 'table')

In [ ]:
# Get the profile of each recipes contained in 'recipes'
def map_recipes(recipes, mapping):
    threshold = 0.3
    profile = []
    mismatches_ratio_in_recipes = []

    for recipe in tqdm.tqdm(recipes):
        nb_ingr = len(recipe)
        mismatches = 0.0
        recipe_mapped = []
        
        for ingredient in recipe:
            if ingredient.startswith('usda_id'):
                usda_id = int(ingredient.split('=')[1])
                res_map = nutrients_mapping.loc[usda_id]
                recipe_mapped.append(res_map.tolist())
            else:
                mismatches += 1
                
        recipe_mapped = pd.DataFrame(recipe_mapped)
        mismatches_ratio_in_recipes.append(mismatches / nb_ingr)
        
        if mismatches / nb_ingr < threshold:
            recipe_mapped = recipe_mapped.sum(axis = 0)
            profile.append(recipe_mapped.tolist())
        else:
            profile.append(list())

    profile = pd.DataFrame(profile)
    profile.columns = mapping.columns
    
    return profile, mismatches_ratio_in_recipes

In [ ]:
# We convert every quantities units to grams, and delete the recipes that contains unconvertable unit
to_remove = []
to_remove_mapping = []
for recipe in tqdm.tqdm_notebook(quantities_recipes):
    for ingr in recipe:
        try:
            ingr[1] = convert_gr[ingr[1]]
            try:
                ingr[2] = mapping_usda_id[clean.clean_ingredient(ingr[2])]
            except KeyError:
                to_remove_mapping.append(recipe)
        except KeyError:
            to_remove.append(recipe)
quantities_recipes = [list(map(lambda x: (x[0] * x[1], x[2]), recipe)) for recipe in tqdm.tqdm_notebook(quantities_recipes) if recipe not in to_remove]

In [ ]:
len(to_remove_mapping)

In [ ]:
quantities_recipes

In [ ]:
# Get median weight for each ingredient
ingredient_weight = []
for recipe in tqdm.tqdm_notebook(quantities_recipes):
    for ingredient in recipe:
        ingredient_weight.append(ingredient)
ingredient_weight = np.array(ingredient_weight)
ingredient_weight_df = pd.DataFrame(ingredient_weight, columns=['weight', 'ingredient'])
ingredient_weight_df = ingredient_weight_df.astype({'weight': float})
median_weight_by_ingredient = ingredient_weight_df.groupby('ingredient').median().reset_index()